In [32]:
import pathlib
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import threading
import multiprocessing as mp
import time 
from tqdm import tqdm_notebook as tqdm
import glob
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}
data_file = '../../data/html_to_download.csv'
save_url = '../../data/'
LOCK = threading.Lock()

In [2]:
df = pd.read_csv(data_file)
stores = np.unique(df.store)

to_download = [(df[df.store==store].url).values for store in stores]

NameError: name 'store' is not defined

In [53]:
def get_pages(stores_links, store):
    pathlib.Path(save_url+store).mkdir(parents=True,exist_ok=True)
    
    invalid_pages = []
    for idx, link in enumerate(stores_links):
        if pathlib.Path(f"{save_url}/{store}/{idx}.html").exists():
            continue
        req = requests.get(link, headers=header)

        if(req.status_code == 200):    
            soup = BeautifulSoup(req.text)
            with open(f"{save_url}/{store}/{idx}.html", "w", encoding='utf-8') as file:
                file.write(str(soup))
        else:
            invalid_pages.append(link)
        
        time.sleep(0.5  + np.random.rand()*2)
            
    return invalid_pages

In [54]:
threads = []
for idx in range(len(stores)):
    thread = threading.Thread(target=get_pages, args=(to_download[idx], stores[idx]), name=f't{idx}')
    thread.start()
    time.sleep(0.2)
    threads.append(thread)
    print("Starts crawler on: {}".format(idx))
    
for thread in tqdm(threads):
    thread.join()
    print("finished thread: {}".format(thread.name))

Starts crawler on: 0
Starts crawler on: 1
Starts crawler on: 2
Starts crawler on: 3
Starts crawler on: 4
Starts crawler on: 5
Starts crawler on: 6
Starts crawler on: 7
Starts crawler on: 8
Starts crawler on: 9


finished thread: t0
finished thread: t1
finished thread: t2
finished thread: t3
finished thread: t4
finished thread: t5
finished thread: t6
finished thread: t7
finished thread: t8
finished thread: t9


In [60]:
for idx, store in enumerate(stores):
    print(store, len(glob.glob(save_url+f'/{stores[idx]}/*')), len(to_download[idx]))


amazon 117 117
avenida 9 24
cissamagazine 18 18
colombo 20 20
havan 7 85
ibyte 42 49
kabum 82 87
magazineluiza 24 24
ricardoeletro 83 89
taqi 32 32


In [56]:
havan_list = ['https://www.havan.com.br/celular-smartphone-moto-one-action-64gb-6-34-motorola-azul-denim/p',
              'https://www.havan.com.br/celular-iphone-7-single-chip-32-gb-4-7-apple-prata/p',
              'https://www.havan.com.br/celular-smartphone-k12-dual-chip-5-7-lg-platinum/p',
              'https://www.havan.com.br/celular-smartphone-zenfone-max-pro-m1-64gb-asus-prata/p',
              'https://www.havan.com.br/celular-smartphone-moto-one-action-64gb-6-34-motorola-azul-denim/p',
              'https://www.havan.com.br/celular-smartphone-moto-e6-plus-6-1-32gb-motorola-cinza/p',
              'https://www.havan.com.br/celular-smartphone-galaxy-note10-6-8-256gb-samsung-prata/p']

In [58]:
get_pages(havan_list, 'havan')

[]